In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.1:
            print("Calc_reward target bonus: ", 500 + 500 * (1 - abs(self.env.state.speed)))
            return 500 + 500 * (1 - abs(self.env.state.speed))
        else:
            reward = 0
            reward += -1
            env.step_pen -= 1
            
            if len(self.env.distances) > 0 and len(self.env.history) > 0:
                gps_start_0 = self.env.history[0].gps_actual[0] - self.env.gps_target[0]
                gps_start_1 = self.env.history[0].gps_actual[1] - self.env.gps_target[1]
                
                start_dist = math.sqrt(gps_start_0**2 + gps_start_1**2)
                
                
                
                target_diff = self.env.distances[-1] - self.env.get_target_distance(False)
                
                gps_diff_0 = self.env.history[-1].gps_actual[0] - self.env.state.gps_actual[0]
                gps_diff_1 = self.env.history[-1].gps_actual[1] - self.env.state.gps_actual[1]
                
                #print("history: ", self.env.history[-1].gps_actual[0])
                #print("actual:  " , self.env.state.gps_actual[0])
                
                actual_diff = math.sqrt(gps_diff_0**2 + gps_diff_1**2)
                
                if actual_diff > 0:
                    diff_rew = abs(target_diff / start_dist * 500) * ( target_diff  / actual_diff)
                else:
                    diff_rew = 0
                    
                                
                env.dist_rew.append(diff_rew)
                if sum(env.dist_rew) > 500:
                    print("Dist_rew too big: ", sum(env.dist_rew))
                    
                env.t_diff.append(target_diff)
                if sum(env.t_diff) > start_dist:
                    print("Sum of t_diff to big: ", sum(env.t_diff))
                    print("start_dist: ",start_dist)
               
                    
                #if diff_rew < 0:
                #    diff_rew = diff_rew * 2
                
                #print("Target_diff: ", target_diff)
                #print("Actual_diff: ", actual_diff)
                #print("start_dist: ", start_dist)
                #print("diff_rew:   ", diff_rew)  
                    
  
                reward += diff_rew
                    
            if self.env.state.action_denied:
                reward += -1
        
            if self.env.state.touching:
                reward += -20
        return reward

    def check_done(self):
        if self.env.total_reward < -500:
            print("reward boundary, reward: ", self.env.total_reward)
            print("              dist_rew : ", sum(env.dist_rew))
            env.dist_rew.clear()
            env.t_diff.clear()
            env.step_pen = 0
            return True
        
        if self.env.step_pen < -300:
            print("step boundary, reward: ", self.env.total_reward)
            print("              dist_rew : ", sum(env.dist_rew))
            env.dist_rew.clear()
            env.t_diff.clear()
            env.step_pen = 0
            return True
        
        if self.env.get_target_distance(False) < 0.1:
            print("target reached, reward: ", self.env.total_reward)
            print("              dist_rew : ", sum(env.dist_rew))
            env.dist_rew.clear()
            env.t_diff.clear()
            env.step_pen = 0
            return True
        
        if self.env.total_reward > 2000:
            print("Stop hacking!, reward: ", self.env.total_reward)
            print("              dist_rew : ", sum(env.dist_rew))
            env.dist_rew.clear()
            env.t_diff.clear()
            env.step_pen = 0
            return True
        
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 5
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
               # action_class = wg.WbtActContinuous(config=config, relative = True),
                action_class = wg.WbtActDiscrete(config = config, dspeed = 0.1, ddir = 0.1, speeds = 3, dirs = 3),
                config=config)

../../backend/webotsgym/env/webotenv.py:123: UserWarning: Relative property of action class is overwritten by config.relative_action. This might interfere with bounds argument for WbtActContinuous.
  warnings.warn("Relative property of action class is overwritten "


Accepting on Port:  10201


In [5]:
env.dist_rew = []
env.t_diff = []
env.step_pen = 0

In [6]:
model_name = "3x3_reach_target_v7_b"
model = PPO1("MlpPolicy", env, timesteps_per_actorbatch = 2000, tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
while True:
    model.learn(total_timesteps=500000)
    model.save("./model_{}".format(model_name))


step boundary, reward:  -229.1583150370377
              dist_rew :  73.84168496296235
step boundary, reward:  -391.7221342757001
              dist_rew :  -71.7221342756998
step boundary, reward:  -217.8341359441237
              dist_rew :  102.1658640558764
step boundary, reward:  -220.6023846625618
              dist_rew :  95.39761533743804
reward boundary, reward:  -500.9176156346374
              dist_rew :  -297.9176156346375
step boundary, reward:  -297.6364856182601
              dist_rew :  9.363514381739677
Calc_reward target bonus:  896.2056711316109
target reached, reward:  1212.634646322709
              dist_rew :  378.42897519109823

step boundary, reward:  -117.13142656148602
              dist_rew :  183.86857343851392
step boundary, reward:  -222.73330516533272
              dist_rew :  83.2666948346672
step boundary, reward:  -302.9411449890617
              dist_rew :  23.05885501093851
step boundary, reward:  -318.79732310574195
              dist_rew :  -1.7973

step boundary, reward:  -116.05722598018644
              dist_rew :  187.9427740198136
step boundary, reward:  -323.8793540367877
              dist_rew :  -5.879354036787801
step boundary, reward:  -264.30199398804075
              dist_rew :  85.69800601195922
step boundary, reward:  -314.4900114520759
              dist_rew :  6.509988547924285
step boundary, reward:  -245.64309547669745
              dist_rew :  90.3569045233024
step boundary, reward:  -432.9531536301496
              dist_rew :  -108.95315363014949
reward boundary, reward:  -500.0093542610953
              dist_rew :  -191.00935426109533
step boundary, reward:  -360.06901113085746
              dist_rew :  -33.06901113085737
step boundary, reward:  -327.8540587807913
              dist_rew :  -7.854058780791525
step boundary, reward:  -375.01355488714916
              dist_rew :  -35.01355488714935
step boundary, reward:  -447.90273781313476
              dist_rew :  -146.90273781313465
step boundary, reward:  -9

step boundary, reward:  -363.587773176546
              dist_rew :  -39.58777317654592
reward boundary, reward:  -510.158333056614
              dist_rew :  -286.15833305661397
reward boundary, reward:  -503.65795153075055
              dist_rew :  -268.6579515307507
step boundary, reward:  -349.22239558408836
              dist_rew :  -43.222395584088055
step boundary, reward:  -361.011705040319
              dist_rew :  -48.01170504031879
step boundary, reward:  -204.40697785017758
              dist_rew :  118.59302214982263
step boundary, reward:  -313.6053019983618
              dist_rew :  3.3946980016383765
Calc_reward target bonus:  792.1793907880783
target reached, reward:  874.0680945036788
              dist_rew :  345.8887037156005
step boundary, reward:  -175.3786439975948
              dist_rew :  162.62135600240524
reward boundary, reward:  -500.0481918219111
              dist_rew :  -193.04819182191133
step boundary, reward:  -370.77158576101573
              dist_rew 

step boundary, reward:  -138.86567438647893
              dist_rew :  171.13432561352099
step boundary, reward:  -87.87305291160845
              dist_rew :  213.12694708839143
step boundary, reward:  -191.32641347546698
              dist_rew :  129.673586524533
step boundary, reward:  -291.70567589311105
              dist_rew :  26.294324106889093
step boundary, reward:  -146.55233561614392
              dist_rew :  161.44766438385636
step boundary, reward:  -259.8439400373608
              dist_rew :  65.15605996263923
step boundary, reward:  -351.9875702193195
              dist_rew :  -37.98757021931943
step boundary, reward:  -262.76160042136956
              dist_rew :  63.238399578630606
step boundary, reward:  -292.537026098803
              dist_rew :  14.462973901197138
step boundary, reward:  -402.0365699711146
              dist_rew :  -90.03656997111466
step boundary, reward:  -360.00125100623194
              dist_rew :  -42.001251006232266
reward boundary, reward:  -50

reward boundary, reward:  -502.38462075640916
              dist_rew :  -335.38462075640905
step boundary, reward:  -146.32652896729232
              dist_rew :  156.67347103270768
step boundary, reward:  -456.942653744074
              dist_rew :  -153.9426537440746
step boundary, reward:  -396.06018598891006
              dist_rew :  -33.060185988909936
step boundary, reward:  -294.5544751209026
              dist_rew :  187.44552487909712
reward boundary, reward:  -500.90821894840866
              dist_rew :  -298.90821894840855
step boundary, reward:  -52.18940791649659
              dist_rew :  252.81059208350334
reward boundary, reward:  -515.561548296365
              dist_rew :  -438.561548296365
step boundary, reward:  -373.9729760090458
              dist_rew :  -1.9729760090454964
step boundary, reward:  -293.4069670145124
              dist_rew :  32.59303298548751
step boundary, reward:  -170.09413143004326
              dist_rew :  161.90586856995668
step boundary, reward

...

step boundary, reward:  -120.44877605347367
              dist_rew :  180.55122394652628
step boundary, reward:  -138.46054403613152
              dist_rew :  163.5394559638685
step boundary, reward:  14.720063770268144
              dist_rew :  315.72006377026815
step boundary, reward:  -151.33616867038398
              dist_rew :  215.66383132961585
step boundary, reward:  -16.285455074824046
              dist_rew :  295.71454492517637
Calc_reward target bonus:  624.8932182788849
target reached, reward:  985.0020706485818
              dist_rew :  414.10885236969705
step boundary, reward:  -180.228456275506
              dist_rew :  120.77154372449417
Calc_reward target bonus:  956.3460610806942
target reached, reward:  1089.8110002964293
              dist_rew :  337.464939215735
step boundary, reward:  -134.58013774820873
              dist_rew :  166.4198622517912
Calc_reward target bonus:  598.7800359725952
target reached, reward:  913.360058402848
              dist_rew :  414.

step boundary, reward:  -209.87189855184886
              dist_rew :  91.12810144815106
step boundary, reward:  -181.20471859796558
              dist_rew :  138.79528140203445
step boundary, reward:  -273.5130031706048
              dist_rew :  85.48699682939512
step boundary, reward:  -115.2202511238592
              dist_rew :  185.77974887614087
step boundary, reward:  -265.01103278796563
              dist_rew :  104.98896721203442
reward boundary, reward:  -501.6626732057582
              dist_rew :  14.337326794241722
Calc_reward target bonus:  962.6797400414944
target reached, reward:  1104.4817658190152
              dist_rew :  405.8020257775205
step boundary, reward:  40.72729540996398
              dist_rew :  343.72729540996386
Calc_reward target bonus:  679.5786619186401
target reached, reward:  1016.7021990036301
              dist_rew :  402.12353708499
Calc_reward target bonus:  608.5348427295685
target reached, reward:  915.286242002732
              dist_rew :  352.7

step boundary, reward:  -153.67502212678497
              dist_rew :  159.32497787321503
step boundary, reward:  -271.5796760326005
              dist_rew :  56.42032396739948
step boundary, reward:  -60.94210557670456
              dist_rew :  243.05789442329555
Calc_reward target bonus:  635.3423595428467
target reached, reward:  983.5003662676186
              dist_rew :  415.15800672477195
step boundary, reward:  -193.8978122467153
              dist_rew :  146.10218775328462
step boundary, reward:  -172.97325542483213
              dist_rew :  248.02674457516787
step boundary, reward:  -121.82249642121158
              dist_rew :  189.1775035787883
Calc_reward target bonus:  894.7277739644051
target reached, reward:  1160.3826903981587
              dist_rew :  419.65491643375367
step boundary, reward:  86.80436313904579
              dist_rew :  387.8043631390455
Calc_reward target bonus:  533.3447456359863
target reached, reward:  839.5083195808588
              dist_rew :  420.

Calc_reward target bonus:  722.6205468177795
target reached, reward:  963.4904961378613
              dist_rew :  472.8699493200815
Calc_reward target bonus:  925.5750104784966
target reached, reward:  1005.6746194579451
              dist_rew :  365.09960897944904
step boundary, reward:  -188.458593286438
              dist_rew :  221.54140671356186
step boundary, reward:  -301.95417605286866
              dist_rew :  20.045823947131424
step boundary, reward:  -262.1712962553768
              dist_rew :  38.828703744623134
step boundary, reward:  -137.6666004959821
              dist_rew :  163.33339950401805
step boundary, reward:  -144.57279749696167
              dist_rew :  156.4272025030382
reward boundary, reward:  -501.1140924884711
              dist_rew :  0.885907511529099
Calc_reward target bonus:  855.4880768060684
target reached, reward:  1005.8061764352362
              dist_rew :  323.31809962916793
step boundary, reward:  -172.87739562862467
              dist_rew :  1

Calc_reward target bonus:  868.3900684118271
target reached, reward:  993.0036765922325
              dist_rew :  271.61360818040555
step boundary, reward:  -157.98624317404114
              dist_rew :  156.01375682595875
Calc_reward target bonus:  734.607070684433
target reached, reward:  1105.8565351280818
              dist_rew :  421.24946444364866
step boundary, reward:  63.580911181038296
              dist_rew :  364.5809111810382
step boundary, reward:  -18.732554543374917
              dist_rew :  325.2674454566253
Calc_reward target bonus:  933.800496160984
target reached, reward:  1300.673124720017
              dist_rew :  405.87262855903305
step boundary, reward:  -12.014979642816062
              dist_rew :  295.98502035718394
Calc_reward target bonus:  527.1930992603302
target reached, reward:  796.0334975726889
              dist_rew :  361.84039831235873
Calc_reward target bonus:  790.1911288499832
target reached, reward:  945.8181022408735
              dist_rew :  35